In [2]:
import hashlib
import random

def get_odd_number(a, b):
    """Generate a random odd number between a and b."""
    rand = int(random.randrange(a, b))
    if rand % 2 == 0:
        rand += 1
    return rand

def is_composite(n, a, r, s):
    """Check if a number n is composite using a base a."""
    res = int(pow(a, r, n))
    if res == 1 or res == n - 1:
        return False
    for _ in range(int(s)):
        res = int(pow(res, 2, n))
        if res == n - 1:
            return False
    return True

def is_prime(n):
    """Check if a number n is prime."""
    if n <= 1:
        return False
    r = int(n) - 1
    s = 0
    while r % 2 == 0:
        r //= 2
        s += 1
    limit = (n - 1) // 4
    for _ in range(int(limit)):
        a = random.randint(2, n - 2)
        if is_composite(n, a, r, s):
            return False
    return True

def generate_prime_number(a, b):
    """Generate a prime number between a and b."""
    while True:
        number = get_odd_number(a, b)
        if is_prime(number):
            return number

def gcd(a, b):
    """Compute the greatest common divisor of a and b."""
    while b != 0:
        a, b = b, a % b
    return a

def multiplicative_inverse(e, phi):
    """Compute the modular multiplicative inverse of e modulo phi."""
    temp_phi = phi
    y = 0
    x = 1
    while e > 1:
        quotient = e // temp_phi
        t = temp_phi
        temp_phi = e % temp_phi
        e = t
        t = y
        y = x - (quotient * y)
        x = t
    if x < 0:
        x += phi
    return x

class Decryptor:
    def __init__(self):
        """Initialize Decryptor with generated prime numbers and keys."""
        p = generate_prime_number(2, 100)
        q = generate_prime_number(2, 100)
        self.n = p * q
        self.phi = (p - 1) * (q - 1)
        self.e = self.generate_public_key()
        self.d = multiplicative_inverse(self.e, self.phi)

    def generate_public_key(self):
        """Generate a public key."""
        public_key = 2
        while public_key < self.phi:
            if gcd(public_key, self.phi) == 1:
                return public_key
            public_key += 1

    def get_public_key(self):
        """Return the public key (n, e)."""
        return self.n, self.e

    def decrypt(self, data):
        """Decrypt data using the private key."""
        return "".join([chr(pow(char, self.d, self.n)) for char in data])

    def sign_message(self, message):
        """Sign a message using the private key."""
        message_hash = hashlib.sha256(message.encode("UTF-8")).hexdigest()
        signature = [(pow(ord(char), self.d, self.n)) for char in message_hash]
        return signature

class Encryptor:
    def __init__(self, n, e):
        """Initialize Encryptor with public key (n, e)."""
        self.n = n
        self.e = e

    def encrypt(self, data):
        """Encrypt data using the public key."""
        return [pow(ord(char), self.e, self.n) for char in data]

    def verify_signature(self, signature, message):
        """Verify the signature of a message."""
        try:
            message_hash = hashlib.sha256(message.encode("UTF-8")).hexdigest()
            signature_hash_value = "".join([chr(pow(char, self.e, self.n)) for char in signature])
            return signature_hash_value == message_hash
        except Exception as e:
            print(f"Error: Signature verification process failed - {e}")
            return False

# Example usage
bob = Decryptor()
print("Public key from Decryptor of Bob:", bob.get_public_key())

alice = Decryptor()
print("Public key from Decryptor of Alice:", alice.get_public_key())

alice_encryptor = Encryptor(*alice.get_public_key())
bob_encryptor = Encryptor(*bob.get_public_key())

bob_message = input("Enter message from Bob here: ")
alice_message = input("Enter message from Alice here: ")

# Encrypt messages
encrypted_bob = alice_encryptor.encrypt(bob_message)
encrypted_alice = bob_encryptor.encrypt(alice_message)

# Print encrypted messages
print("Encrypted message from Bob:", encrypted_bob)
print("Encrypted message from Alice:", encrypted_alice)

# Sign messages
bob_signature = bob.sign_message(bob_message)

# Decrypt messages
decrypted_bob = alice.decrypt(encrypted_bob)

if bob_encryptor.verify_signature(bob_signature, decrypted_bob):
    print("Decrypted Message is from Bob:")
    print(decrypted_bob)

alice_signature = alice.sign_message(alice_message)
decrypted_alice = bob.decrypt(encrypted_alice)

if alice_encryptor.verify_signature(alice_signature, decrypted_alice):
    print("Decrypted Message is from Alice:")
    print(decrypted_alice)


Public key from Decryptor of Bob: (553, 5)
Public key from Decryptor of Alice: (5293, 5)


Enter message from Bob here:  i am moon
Enter message from Alice here:  star


Encrypted message from Bob: [566, 2105, 3522, 1970, 2105, 1970, 3764, 3764, 4212]
Encrypted message from Alice: [208, 443, 125, 305]
Decrypted Message is from Bob:
i am moon
Decrypted Message is from Alice:
star
